In [ ]:
! pip install polars seaborn gdown

In [ ]:
! rm titanic_train.csv && gdown --id 10W80Ylshf2hJGzK9u1Ita7cfxrEscjkX

# BastionLab Analytics Demo

BastionLab allows data scientists to run queries on data frames remotely without seeing the original data or intermediary results.
The user uses polars' lazy API augmented with BasionLab's objects to define a *Composite Plan* which is an abstract object representing all the instructions to be run on the server in a lazy fashion. BastionLab's *Composite Plan* supports most polars operations from selects to groupbys to joins.

We intentionnaly forbid map operations with user defined functions to avoid remote code execution. As a replacement, we provide the `apply_udf` method on our `RemoteLazyFrame` type that allows the user to apply a custom TorchScript compatible function on a subset of series. Internally, the provided plain python function is compiled into TorchScript and applied on the server to the matching series by casting them as libtorch tensors.

## Data Loading

We load the csv dataset as a plain polars dataframe, we open a connection to the server and we upload the data frame.
The server sends back some metadata (a reference and the schema of the data frame) that are wrapped in a `FetchableLazyFrame`.
`FetchableLazyFrame`s are a subtype of `RemoteLazyFrames` that can be used not only to define the computation graph by using polars' lazy API but also to retrieve the data (we'll add a mechanism to enforce security and privacy rules in the future).

In [ ]:
import polars as pl
from bastionlab import Connection

df = pl.read_csv("titanic_train.csv").limit(50)

connection = Connection('localhost', 50056)
client = connection.client

rdf = client.send_df(df)

rdf

The `RemoteLazyFrame`s (and their subtypes) can be used to query the metadata without performing any actual request to the server.

In [ ]:
rdf.columns

The user may also invoke polars' lazy API to construct query plans.

In the following cell, we compute the survival rates of the passengers of the Titanic with respect to their ticket class.

Note that issuing such aggregated queries without seing the original dataset or intermediary results already provides some sense of privacy. We plan to add a budgeting mechanism such as Differential Privacy to allow the data owner to control how their data can be used.

In [ ]:
per_class_rates = (
        rdf.select([pl.col("Pclass"), pl.col("Survived")])
        .groupby(pl.col("Pclass"))
        .agg(pl.col("Survived").mean())
        .sort("Survived", reverse=True)
        .collect()
        .fetch()
    )

per_class_rates

We can do the same with respect to the sex of the passengers.

In [ ]:
per_sex_rates = (
        rdf.select([pl.col("Sex"), pl.col("Survived")])
        .groupby(pl.col("Sex"))
        .agg(pl.col("Survived").mean())
        .sort("Survived", reverse=True)
        .collect()
        .fetch()
    )

per_sex_rates

Aggregated queries may be used locally to make plots

In [ ]:
import seaborn as sns
import torch
from bastionlab import RemoteLazyFrame

def bar_plot(rdf: RemoteLazyFrame, col_x: str, col_y: str) -> RemoteLazyFrame:
    def f(x):
        bins = 10 * torch.ones_like(x)
        return x // bins * bins

    data = (
        rdf
        .select(
            [pl.col(col_x), pl.col(col_y)]
        )
        .apply_udf([col_x], f)
        .groupby(pl.col(col_x))
        .agg(pl.col(col_y).count())
        .sort(col_x)
        .collect()
        .fetch()
    )
    sns.barplot(x=col_x, y=col_y, data=data.to_pandas())

rdf = rdf.filter(pl.col("Age") != None)
bp = bar_plot(rdf, "Age", "Survived")

We finally close the conection to the server.

In [ ]:
connection.close()